<div style="text-align: center">
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/shaowei.jpeg" alt="ShaoWei" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/heart.png" alt="❤️" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/julia.png" alt="Julia" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/Meme-like-a-boss.jpg" alt="like a bozz" />
</div>

<div style="font-size:48px; text-align: center">
Walk Like Python; Run Like C
</div>

## Easy to write what I meant

Julia expressions are evaluated on-the-fly, like interpreted languages.

In [1]:
println("Hello, World!")

Hello, World!


String interpolation works.

In [2]:
who_we_are = "MOE ESTL"
println("Hello, we are $(who_we_are[5:end])!")

Hello, we are ESTL!


Functions are JIT compiled. Typing is optional.

In [109]:
function double(x)
    return 2x
end

double (generic function with 2 methods)

In [110]:
# Equivalently
double(x) = 2x

double (generic function with 2 methods)

Functions can also be overloaded using specialisation.

In [111]:
double(s::Vector) = [s; s]

double (generic function with 2 methods)

In [6]:
@show double(5)
@show double(5.0)
@show double([4, 5, 6])
@show double("dragon")

double(5) = 10
double(5.0) = 10.0
double([4, 5, 6]) = [4, 5, 6, 4, 5, 6]


LoadError: [91mMethodError: no method matching *(::Int64, ::String)[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Real, [91m::Complex{Bool}[39m) at complex.jl:254
  *(::Real, [91m::Complex[39m) at complex.jl:266
  ...[39m

Let's see what happened if we define a function without a catch-all version.

In [7]:
echo(s::String) = (s * "!")^3

echo (generic function with 1 method)

In [8]:
@show echo("slam")
@show echo("8")
@show echo(:slam)

echo("slam") = "slam!slam!slam!"
echo("8") = "8!8!8!"


LoadError: [91mMethodError: no method matching echo(::Symbol)[0m
Closest candidates are:
  echo([91m::String[39m) at In[7]:1[39m

## Have C and Python binding

Interfacing with shared libraries written in C is straightforward. For example, say we have a `libhello.so` created from [hello.c](hello.c).

In [112]:
ccall((:hello, "libhello.so"), Void, (Cstring,), "Moto")

Hello, Moto!


Calling Julia functions from C can be done by including `julia.h` that can be found inside the Julia installation (hereby handwaved as it's outside the scope of this sharing).

Julia can call Python objects using the `PyCall.jl` library.

In [10]:
using PyCall

In [11]:
py"map(lambda x: 2 * x, [1, 2, 3, 4])"

4-element Array{Int64,1}:
 2
 4
 6
 8

In [12]:
@pycall PyObject(map)(x -> 2x, [1, 2, 3, 4])::Vector{Int}

4-element Array{Int64,1}:
 2
 4
 6
 8

Calling Julia functions from Python can be done using the Python library `PyJulia` (hereby handwaved again as it's outside the scope of this sharing).

## Can pop open the hood

Methods introspection - See intermediate and compiled representations.

In [13]:
methods(double)

# 2 methods for generic function "double":
double(s::Array{T,1} where T) in Main at In[5]:1
double(x) in Main at In[4]:2

In [14]:
@code_typed double(5)

CodeInfo(:(begin 
        return (Base.mul_int)(2, x)::Int64
    end))=>Int64

In [15]:
@code_typed double(5.0)

CodeInfo(:(begin 
        return (Base.mul_float)((Base.sitofp)(Float64, 2)::Float64, x)::Float64
    end))=>Float64

In [16]:
@code_typed double([4, 5, 6])

CodeInfo(:(begin 
        return $(Expr(:invoke, MethodInstance for vcat(::Array{Int64,1}, ::Array{Int64,1}), :(Main.vcat), :(s), :(s)))
    end))=>Array{Int64,1}

In [17]:
@code_native double(5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	leaq	(%rdi,%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [18]:
@code_native double(5.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	addsd	%xmm0, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Constant function calls in a function definition are run during compilation for speedups.

In [113]:
boyband_song(x) = double(5) + double(6) + x

boyband_song (generic function with 1 method)

In [114]:
@code_native boyband_song(5566)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[113]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	22(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Example: Two ways to write the same function...

In [19]:
function task_1()
    result = 0
    for i in 1:100000
        result += 1/(i^2)
    end
    return result
end

task_1 (generic function with 1 method)

In [20]:
function task_2()
    result = 0.0
    for i in 1:100000
        result += 1/(i^2)
    end
    return result
end

task_2 (generic function with 1 method)

Let's time it

In [21]:
task_1() #First time is to compile the method once.
@time task_1()

  0.004609 seconds (300.08 k allocations: 4.584 MiB)


1.6449240668982423

In [22]:
task_2() #First time is to compile the method once.
@time task_2()

  0.000459 seconds (5 allocations: 176 bytes)


1.6449240668982423

What happened?

In [23]:
@code_warntype task_1()

Variables:
  #self# <optimized out>
  i::Int64
  #temp#@_3::Int64
  result::Union{Float64, Int64}
  #temp#@_5::Core.MethodInstance
  #temp#@_6::Float64

Body:
  begin 
      result::Union{Float64, Int64} = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, 100000)::Bool, 100000, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#@_3::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#@_3::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 30
      SSAValue(3) = #temp#@_3::Int64
      SSAValue(4) = (Base.add_int)(#temp#@_3::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#@_3::Int64 = SSAValue(4) # line 4:
      unless (result::Union{Float64, Int64} isa Int64)::Bool goto 15
      #temp#@_5::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      15: 
      unless (result::Union{Float64, Int64} isa Float64)::Bool goto 19
      #temp#@_5::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
 

In [24]:
@code_warntype task_2()

Variables:
  #self# <optimized out>
  i::Int64
  #temp#::Int64
  result::Float64

Body:
  begin 
      result::Float64 = 0.0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, 100000)::Bool, 100000, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 15
      SSAValue(3) = #temp#::Int64
      SSAValue(4) = (Base.add_int)(#temp#::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#::Int64 = SSAValue(4) # line 4:
      result::Float64 = (Base.add_float)(result::Float64, (Base.div_float)((Base.sitofp)(Float64, 1)::Float64, (Base.sitofp)(Float64, (Base.mul_int)(i::Int64, i::Int64)::Int64)::Float64)::Float64)::Float64
      13: 
      goto 5
      15:  # line 6:
      return result::Float64
  end::Float64


Type introspection - work with the relationship between types.

In [25]:
typeof(5)

Int64

In [26]:
Int64 <: Integer

true

In [27]:
is_an_integer(x::Integer) = true
is_an_integer(x) = false

is_an_integer (generic function with 2 methods)

In [28]:
@show is_an_integer(5)
@show is_an_integer(5.0)

is_an_integer(5) = true
is_an_integer(5.0) = false


false

Strong reflection makes it easy and fast to implement much of Julia in Julia.

In [29]:
is_type{T}(x::T, t::Type{T}) = true
is_type(x, t) = false

is_type (generic function with 2 methods)

In [30]:
@show is_type(5.0, Float64)
@show is_type(5, Int)
@show is_type(5, String)
@show is_type(is_type, Function)

is_type(5.0, Float64) = true
is_type(5, Int) = true
is_type(5, String) = false
is_type(is_type, Function) = true


true

Reflection

In [128]:
@show double(6)
@show double(7.5)

double(6) = 12
double(7.5) = 15.0


15.0

In [135]:
input(text::String) = (print("$(text): "); return readline())

function run_function_on_demand()
    f = Symbol(input("Function to run"))
    v = input("what's the numerical value")
    t = Symbol(input("what's the value's type"))
    eval(:($f(parse($t, $v))))
end

run_function_on_demand()

Function to run: STDIN> double
what's the numerical value: STDIN> 6
what's the value's type: STDIN> Float64


12.0

In [33]:
struct QMInt
    value::Int
end

In [34]:
Base.:+(x::QMInt, y::QMInt) = QMInt(x.value + y.value)
Base.:-(x::QMInt, y::QMInt) = QMInt(x.value - y.value)
function Base.show(io::IO, x::QMInt)
    text = x.value == 3 ? "3, quick maffs" : "$(x.value)"
    show(io, text)
end

In [35]:
two = QMInt(2); one = QMInt(1)
@show what = two + two;
@show what - one;

what = two + two = "4"
what - one = "3, quick maffs"


## Malleable and expressive

In [136]:
v = [2, 4, 6, 8, 10]
@show v[1]
@show v[2]
v[2] = 100
@show v;

v[1] = 2
v[2] = 4
v = [2, 100, 6, 8, 10]


5-element Array{Int64,1}:
   2
 100
   6
   8
  10

Another example of reflection. Don't like Julia's 1-indexing? Create and use your own vector.

In [36]:
struct MyVector{T}
    data::Vector{T}
end
function Base.setindex!(v::MyVector, x, i)
    v.data[i + 1] = x
end

Base.getindex(v::MyVector, i) = v.data[i + 1]

In [37]:
v = MyVector([2, 4, 6, 8, 10])

MyVector{Int64}([2, 4, 6, 8, 10])

In [38]:
@show v[1]
@show v[2]
v[2] = 100
@show v;

v[1] = 4
v[2] = 6
v = MyVector{Int64}([2, 4, 100, 8, 10])


MyVector{Int64}([2, 4, 100, 8, 10])

It can perform for-loops correctly too!

In [39]:
Base.start(v::MyVector) = length(v.data) - 1
Base.done(v::MyVector, state) = state < 0
Base.next(v::MyVector, state) = (v[state], state - 1)

In [40]:
for elem in v
    println(elem)
end

10
8
100
4
2


In [41]:
sum(v)

124

In [140]:
(a == 1) && (a == 2) && (a == 3)

true

In [139]:
type IncrementalInt
    value::Int
end
function Base.:(==)(i::IncrementalInt, rhs)
    i.value += 1
    return i.value == rhs
end
a = IncrementalInt(0)

IncrementalInt(0)

## Supports Unicode operators

In [44]:
4 / 3

1.3333333333333333

In [45]:
4 // 3

4//3

In [46]:
div(4, 3)

1

In [47]:
4 ÷ 3

1

Operators are not special constructs. They are just syntactic sugar for the corresponding function.

In [48]:
÷(4, 3)

1

In [49]:
Base.:÷(4, 3)

1

In [50]:
typeof(:÷)

Symbol

In [51]:
π

π = 3.1415926535897...

In [52]:
α₁ = 3.0; α₂ = 4.0
√(α₁^2 + α₂^2)

5.0

In [53]:
fib(n) = (n ≤ 1) ? n : fib(n - 1) + fib(n - 2)

fib (generic function with 1 method)

In [54]:
@time fib(40)
@time fib(40)
@time fib(40)

  0.697451 seconds (505 allocations: 28.762 KiB)
  0.681746 seconds (5 allocations: 176 bytes)
  0.675590 seconds (5 allocations: 176 bytes)


102334155

## Good for data processing

Julia has Vector that are specialised to the type, implemented as efficiently as possible.

In [57]:
like_a_c_array = [5, 5, 6, 6]

4-element Array{Int64,1}:
 5
 5
 6
 6

In [56]:
a = [1, "is", sum, "number"]

4-element Array{Any,1}:
 1        
  "is"    
  sum     
  "number"

In [55]:
foods_i_like = ["Pasta", "茶そば", "Chicken Pea", "Starbucks"]

4-element Array{String,1}:
 "Pasta"      
 "茶そば"        
 "Chicken Pea"
 "Starbucks"  

Vector is just a 1-D array (`Vector{T}` is alias for `Array{T, 1}`). Julia talks naturally in n-D.

In [58]:
hcat(like_a_c_array, a, foods_i_like)

4×3 Array{Any,2}:
 5  1          "Pasta"      
 5   "is"      "茶そば"        
 6   sum       "Chicken Pea"
 6   "number"  "Starbucks"  

Common to compiled languages, there is quick for-loop, and so there is not a need to vectorise for performance.

In [59]:
for food in foods_i_like
    println("I like $(lowercase(food)) oh yeah~~")
end

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


Common to interpreted languages, functions are first class object. So we have a powerful `do` construct.

In [60]:
foreach(foods_i_like) do food
    println("I like $(lowercase(food)) oh yeah~~")
end

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


In [61]:
foreach(food -> println("I like $(lowercase(food)) oh yeah~~"), foods_i_like)

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


Many approach to data processing.

In [62]:
foods = ["Prata", "油条", "Chicken Rice", "Starbucks"];

In [63]:
map(food -> food ∈ foods_i_like, foods)

4-element Array{Bool,1}:
 false
 false
 false
  true

In [64]:
filter(food -> food ∈ foods_i_like, foods)

1-element Array{String,1}:
 "Starbucks"

In [65]:
map(foods) do food
    return food ∈ foods_i_like
end

4-element Array{Bool,1}:
 false
 false
 false
  true

In [66]:
filter(foods) do food
    return food ∈ foods_i_like
end

1-element Array{String,1}:
 "Starbucks"

In [67]:
[food ∈ foods_i_like for food in foods]

4-element Array{Bool,1}:
 false
 false
 false
  true

In [68]:
[food for food in foods if food ∈ foods_i_like]

1-element Array{String,1}:
 "Starbucks"

A reason to do vectorisation is for code clarity over speed. Julia has a speedy way (both to code and to run) to do that, using syntactic loop fusion.

In [69]:
f(x) = 3x^2 + 5x + 2

f (generic function with 1 method)

In [147]:
X = collect(0.001:0.002:1.0)
function manual_map!(X)
    for i in eachindex(X)
        X[i] = f(2X[i]^2 + 6X[i]^3 - sqrt(X[i]))
    end
end
manual_map!(X) # First run to compile the function.
@time manual_map!(X)
sum(1 ./ X)

  0.000004 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [148]:
X = collect(0.001:0.002:1.0)
function vec_map_1!(X)
    X .= f.(2 .* X.^2 .+ 6 .* X.^3 .- sqrt.(X))
    return
end
vec_map_1!(X) # First run to compile the function.
@time vec_map_1!(X)
sum(1 ./ X)

  0.000004 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [149]:
X = collect(0.001:0.002:1.0)
function vec_map_2!(X)
    @. X = f(2X^2 + 6X^3 - sqrt(X))
    return
end
vec_map_2!(X) # First run to compile the function.
@time vec_map_2!(X)
sum(1 ./ X)

  0.000004 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [150]:
X = collect(0.001:0.002:1.0)
h(x) = 2x^2 + 6x^3 - √x
function vec_map_3!(X)
    X .= ((f ∘ h).(X))
    return
end
vec_map_3!(X) # First run to compile the function.
@time vec_map_3!(X)
sum(1 ./ X)

  0.000004 seconds (4 allocations: 160 bytes)


2.7665013801459013

Plotting is straightforward using [PLots.jl](https://github.com/JuliaPlots/Plots.jl). We can choose which plotting backend (e.g. matplotlib, GR, PlotlyJS) to use due to Julia's reflection.

In [74]:
data = readcsv("ex1data1.txt");

In [105]:
using Plots
gr() # Use the GR plotting library instead

function plot_data(X, y)
    scatter(X, y, marker = (:cross, :red))
end

X = data[:, 1]; y = data[:, 2];
m = length(y);
plot_data(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

We can easily convert mathematical equations into code, which is a daily challenge for data scientist.

Say, we have $y \in \mathbb{R}_m$, $X \in \mathbb{R}_{m, n}$

Firstly, prepend the intercept column to $X$, i.e. $X \rightarrow [0_m \ X]$

Cost function: $J_{X, y}(\theta) = \dfrac{1}{2m} (X \theta - y)^T (X \theta - y) = \dfrac{1}{2m} (X \theta - y) \cdot (X \theta - y)$

Cost gradient: $\partial J_{X, y} (\theta) = \dfrac{1}{m} X^T (X \theta - y)$

For a given $\alpha$ and $\theta$, run $\theta \rightarrow \theta - \alpha \ \partial J_{X, y}(\theta) = \theta - \dfrac{\alpha}{m} X^T (X \theta - y)$ for given number of iterations to approach optimal $\theta$.

In [76]:
prepend_intercept(A) = [ones(eltype(A), size(A, 1)) A]

prepend_intercept (generic function with 1 method)

In [77]:
function make_cost_functions(X, y)
    m = length(y)
    
    function J(θ)
        δ = X * θ .- y
        return (δ ⋅ δ) / (2m)
    end
    
    function ∂J(θ)
        δ = X * θ .- y
        return (X' * δ) / m
    end
    return J, ∂J
end

make_cost_functions (generic function with 1 method)

In [78]:
function run_gradient_descent(X, y, θ, α, iterations)
    X = prepend_intercept(X)

    J, ∂J = make_cost_functions(X, y)
    
    println("Initial: θ = $θ; Cost == $(J(θ))")
    for i in 1:iterations
        θ = θ - α * ∂J(θ)
        println("Iteration $i: θ = $θ; Cost == $(J(θ)).")
    end
    
    return θ
end

run_gradient_descent (generic function with 1 method)

In [79]:
α = 0.01
θ = [0, 0]
iterations = 1500
θ = run_gradient_descent(X, y, θ, α, iterations)

@show θ; # Expected θ ≈ [-3.6303, 1.1664]

Initial: θ = [0, 0]; Cost == 32.072733877455676
Iteration 1: θ = [0.0583914, 0.653288]; Cost == 6.737190464870009.
Iteration 2: θ = [0.0628918, 0.77001]; Cost == 5.931593568604956.
Iteration 3: θ = [0.0578229, 0.791348]; Cost == 5.901154707081388.
Iteration 4: θ = [0.0510636, 0.79573]; Cost == 5.895228586444221.
Iteration 5: θ = [0.0440144, 0.797096]; Cost == 5.890094943117332.
Iteration 6: θ = [0.0369241, 0.797925]; Cost == 5.885004158443646.
Iteration 7: θ = [0.0298371, 0.798658]; Cost == 5.8799324804914175.
Iteration 8: θ = [0.0227612, 0.799373]; Cost == 5.874879094762575.
Iteration 9: θ = [0.0156977, 0.800083]; Cost == 5.869843911806386.
Iteration 10: θ = [0.0086469, 0.800791]; Cost == 5.864826865312929.
Iteration 11: θ = [0.00160879, 0.801499]; Cost == 5.859827889932181.
Iteration 12: θ = [-0.00541662, 0.802204]; Cost == 5.85484692057229.
Iteration 13: θ = [-0.0124294, 0.802909]; Cost == 5.849883892376587.
Iteration 14: θ = [-0.0194295, 0.803612]; Cost == 5.844938740722034.
Iterat

Iteration 151: θ = [-0.868345, 0.888895]; Cost == 5.311380876611356.
Iteration 152: θ = [-0.873803, 0.889443]; Cost == 5.3083753817430726.
Iteration 153: θ = [-0.87925, 0.88999]; Cost == 5.305380712493861.
Iteration 154: θ = [-0.884688, 0.890537]; Cost == 5.302396829870465.
Iteration 155: θ = [-0.890115, 0.891082]; Cost == 5.2994236950200815.
Iteration 156: θ = [-0.895533, 0.891626]; Cost == 5.296461269229852.
Iteration 157: θ = [-0.900942, 0.89217]; Cost == 5.29350951392636.
Iteration 158: θ = [-0.90634, 0.892712]; Cost == 5.290568390675129.
Iteration 159: θ = [-0.911729, 0.893253]; Cost == 5.287637861180118.
Iteration 160: θ = [-0.917108, 0.893794]; Cost == 5.284717887283231.
Iteration 161: θ = [-0.922477, 0.894333]; Cost == 5.281808430963812.
Iteration 162: θ = [-0.927837, 0.894871]; Cost == 5.278909454338152.
Iteration 163: θ = [-0.933187, 0.895409]; Cost == 5.276020919659.
Iteration 164: θ = [-0.938527, 0.895945]; Cost == 5.27314278931507.
Iteration 165: θ = [-0.943858, 0.896481];

Iteration 300: θ = [-1.58199, 0.960588]; Cost == 4.964362046184744.
Iteration 301: θ = [-1.58616, 0.961007]; Cost == 4.962606493117519.
Iteration 302: θ = [-1.59033, 0.961426]; Cost == 4.96085726345113.
Iteration 303: θ = [-1.59448, 0.961843]; Cost == 4.959114334409053.
Iteration 304: θ = [-1.59863, 0.96226]; Cost == 4.957377683296804.
Iteration 305: θ = [-1.60277, 0.962676]; Cost == 4.955647287501639.
Iteration 306: θ = [-1.6069, 0.963091]; Cost == 4.95392312449227.
Iteration 307: θ = [-1.61103, 0.963506]; Cost == 4.95220517181856.
Iteration 308: θ = [-1.61515, 0.963919]; Cost == 4.95049340711124.
Iteration 309: θ = [-1.61926, 0.964332]; Cost == 4.94878780808161.
Iteration 310: θ = [-1.62336, 0.964745]; Cost == 4.947088352521258.
Iteration 311: θ = [-1.62746, 0.965156]; Cost == 4.94539501830176.
Iteration 312: θ = [-1.63155, 0.965567]; Cost == 4.943707783374398.
Iteration 313: θ = [-1.63563, 0.965977]; Cost == 4.942026625769877.
Iteration 314: θ = [-1.6397, 0.966386]; Cost == 4.940351

Iteration 450: θ = [-2.1306, 1.0157]; Cost == 4.760637879102053.
Iteration 451: θ = [-2.13378, 1.01602]; Cost == 4.7596161287279255.
Iteration 452: θ = [-2.13696, 1.01634]; Cost == 4.75859805863968.
Iteration 453: θ = [-2.14013, 1.01666]; Cost == 4.757583655581141.
Iteration 454: θ = [-2.14329, 1.01698]; Cost == 4.7565729063438775.
Iteration 455: θ = [-2.14645, 1.01729]; Cost == 4.755565797767038.
Iteration 456: θ = [-2.14961, 1.01761]; Cost == 4.7545623167371724.
Iteration 457: θ = [-2.15275, 1.01793]; Cost == 4.753562450188067.
Iteration 458: θ = [-2.15589, 1.01824]; Cost == 4.75256618510057.
Iteration 459: θ = [-2.15903, 1.01856]; Cost == 4.751573508502425.
Iteration 460: θ = [-2.16216, 1.01887]; Cost == 4.750584407468098.
Iteration 461: θ = [-2.16529, 1.01919]; Cost == 4.7495988691186195.
Iteration 462: θ = [-2.16841, 1.0195]; Cost == 4.7486168806214.
Iteration 463: θ = [-2.17152, 1.01981]; Cost == 4.747638429190077.
Iteration 464: θ = [-2.17463, 1.02013]; Cost == 4.746663502084345

Iteration 602: θ = [-2.55398, 1.05824]; Cost == 4.6408810713009245.
Iteration 603: θ = [-2.5564, 1.05848]; Cost == 4.64029067803744.
Iteration 604: θ = [-2.55881, 1.05872]; Cost == 4.63970241133642.
Iteration 605: θ = [-2.56122, 1.05896]; Cost == 4.6391162635381065.
Iteration 606: θ = [-2.56363, 1.0592]; Cost == 4.638532227010338.
Iteration 607: θ = [-2.56603, 1.05945]; Cost == 4.637950294148439.
Iteration 608: θ = [-2.56843, 1.05969]; Cost == 4.637370457375125.
Iteration 609: θ = [-2.57082, 1.05993]; Cost == 4.636792709140407.
Iteration 610: θ = [-2.57321, 1.06017]; Cost == 4.636217041921488.
Iteration 611: θ = [-2.57559, 1.06041]; Cost == 4.635643448222671.
Iteration 612: θ = [-2.57797, 1.06065]; Cost == 4.635071920575256.
Iteration 613: θ = [-2.58035, 1.06088]; Cost == 4.634502451537443.
Iteration 614: θ = [-2.58272, 1.06112]; Cost == 4.633935033694242.
Iteration 615: θ = [-2.58509, 1.06136]; Cost == 4.633369659657366.
Iteration 616: θ = [-2.58745, 1.0616]; Cost == 4.632806322065145

Iteration 754: θ = [-2.87581, 1.09057]; Cost == 4.571682552394068.
Iteration 755: θ = [-2.87765, 1.09075]; Cost == 4.5713414082023025.
Iteration 756: θ = [-2.87949, 1.09094]; Cost == 4.571001492792228.
Iteration 757: θ = [-2.88132, 1.09112]; Cost == 4.5706628017378454.
Iteration 758: θ = [-2.88315, 1.0913]; Cost == 4.570325330629095.
Iteration 759: θ = [-2.88497, 1.09149]; Cost == 4.569989075071803.
Iteration 760: θ = [-2.8868, 1.09167]; Cost == 4.569654030687624.
Iteration 761: θ = [-2.88862, 1.09185]; Cost == 4.56932019311398.
Iteration 762: θ = [-2.89043, 1.09204]; Cost == 4.568987558004012.
Iteration 763: θ = [-2.89224, 1.09222]; Cost == 4.568656121026514.
Iteration 764: θ = [-2.89405, 1.0924]; Cost == 4.568325877865882.
Iteration 765: θ = [-2.89586, 1.09258]; Cost == 4.567996824222056.
Iteration 766: θ = [-2.89766, 1.09276]; Cost == 4.567668955810466.
Iteration 767: θ = [-2.89946, 1.09294]; Cost == 4.567342268361973.
Iteration 768: θ = [-2.90126, 1.09312]; Cost == 4.56701675762281

Iteration 921: θ = [-3.14116, 1.11722]; Cost == 4.5288144689118335.
Iteration 922: θ = [-3.14252, 1.11736]; Cost == 4.528627733082892.
Iteration 923: θ = [-3.14387, 1.1175]; Cost == 4.528441669865631.
Iteration 924: θ = [-3.14523, 1.11763]; Cost == 4.528256276837342.
Iteration 925: θ = [-3.14658, 1.11777]; Cost == 4.52807155158404.
Iteration 926: θ = [-3.14793, 1.1179]; Cost == 4.527887491700442.
Iteration 927: θ = [-3.14928, 1.11804]; Cost == 4.527704094789922.
Iteration 928: θ = [-3.15063, 1.11817]; Cost == 4.527521358464489.
Iteration 929: θ = [-3.15197, 1.11831]; Cost == 4.527339280344757.
Iteration 930: θ = [-3.15331, 1.11844]; Cost == 4.527157858059904.
Iteration 931: θ = [-3.15465, 1.11858]; Cost == 4.52697708924765.
Iteration 932: θ = [-3.15599, 1.11871]; Cost == 4.5267969715542264.
Iteration 933: θ = [-3.15732, 1.11885]; Cost == 4.52661750263434.
Iteration 934: θ = [-3.15865, 1.11898]; Cost == 4.526438680151146.
Iteration 935: θ = [-3.15998, 1.11911]; Cost == 4.526260501776217

Iteration 1072: θ = [-3.32112, 1.1353]; Cost == 4.507035919842457.
Iteration 1073: θ = [-3.32215, 1.13541]; Cost == 4.506927629090445.
Iteration 1074: θ = [-3.32319, 1.13551]; Cost == 4.506819728395472.
Iteration 1075: θ = [-3.32422, 1.13561]; Cost == 4.506712216352576.
Iteration 1076: θ = [-3.32525, 1.13572]; Cost == 4.506605091561853.
Iteration 1077: θ = [-3.32628, 1.13582]; Cost == 4.5064983526284434.
Iteration 1078: θ = [-3.32731, 1.13592]; Cost == 4.506391998162513.
Iteration 1079: θ = [-3.32833, 1.13603]; Cost == 4.506286026779232.
Iteration 1080: θ = [-3.32935, 1.13613]; Cost == 4.506180437098759.
Iteration 1081: θ = [-3.33037, 1.13623]; Cost == 4.506075227746218.
Iteration 1082: θ = [-3.33139, 1.13633]; Cost == 4.505970397351695.
Iteration 1083: θ = [-3.33241, 1.13644]; Cost == 4.505865944550205.
Iteration 1084: θ = [-3.33343, 1.13654]; Cost == 4.505761867981676.
Iteration 1085: θ = [-3.33444, 1.13664]; Cost == 4.505658166290941.
Iteration 1086: θ = [-3.33545, 1.13674]; Cost ==

Iteration 1221: θ = [-3.45658, 1.14891]; Cost == 4.494532511333592.
Iteration 1222: θ = [-3.45737, 1.14899]; Cost == 4.494469257137467.
Iteration 1223: θ = [-3.45816, 1.14907]; Cost == 4.494406230779307.
Iteration 1224: θ = [-3.45895, 1.14915]; Cost == 4.494343431438454.
Iteration 1225: θ = [-3.45974, 1.14923]; Cost == 4.494280858297201.
Iteration 1226: θ = [-3.46053, 1.14931]; Cost == 4.49421851054079.
Iteration 1227: θ = [-3.46131, 1.14939]; Cost == 4.4941563873574015.
Iteration 1228: θ = [-3.46209, 1.14947]; Cost == 4.494094487938134.
Iteration 1229: θ = [-3.46288, 1.14954]; Cost == 4.494032811477003.
Iteration 1230: θ = [-3.46366, 1.14962]; Cost == 4.493971357170926.
Iteration 1231: θ = [-3.46443, 1.1497]; Cost == 4.493910124219711.
Iteration 1232: θ = [-3.46521, 1.14978]; Cost == 4.493849111826053.
Iteration 1233: θ = [-3.46599, 1.14986]; Cost == 4.4937883191955175.
Iteration 1234: θ = [-3.46676, 1.14993]; Cost == 4.493727745536528.
Iteration 1235: θ = [-3.46754, 1.15001]; Cost ==

Iteration 1370: θ = [-3.56011, 1.15931]; Cost == 4.487229089394611.
Iteration 1371: θ = [-3.56072, 1.15937]; Cost == 4.487192141702837.
Iteration 1372: θ = [-3.56132, 1.15943]; Cost == 4.487155327094518.
Iteration 1373: θ = [-3.56192, 1.15949]; Cost == 4.487118645090292.
Iteration 1374: θ = [-3.56253, 1.15955]; Cost == 4.487082095212529.
Iteration 1375: θ = [-3.56313, 1.15961]; Cost == 4.487045676985317.
Iteration 1376: θ = [-3.56373, 1.15968]; Cost == 4.487009389934457.
Iteration 1377: θ = [-3.56433, 1.15974]; Cost == 4.486973233587461.
Iteration 1378: θ = [-3.56492, 1.1598]; Cost == 4.486937207473539.
Iteration 1379: θ = [-3.56552, 1.15986]; Cost == 4.486901311123601.
Iteration 1380: θ = [-3.56611, 1.15992]; Cost == 4.486865544070243.
Iteration 1381: θ = [-3.56671, 1.15997]; Cost == 4.486829905847748.
Iteration 1382: θ = [-3.5673, 1.16003]; Cost == 4.486794395992074.
Iteration 1383: θ = [-3.56789, 1.16009]; Cost == 4.486759014040852.
Iteration 1384: θ = [-3.56848, 1.16015]; Cost == 4

Ending this section, a new v0.7 feature that I find interesting and further simplify representation for data science.

In [80]:
# Coming in v0.7
distance((x₁, y₁), (x₂, y₂)) = √((y₂ - y₁)^2 + (x₂, x₁)^2)

distance((0, 3), (4, 0)) # will returns 5.0

LoadError: [91msyntax: "(x₁,y₁)" is not a valid function argument name[39m

In [81]:
typeof((1, 2, "3"))

Tuple{Int64,Int64,String}

## Meta-programming

Like Lisp, we can write macros like functions, which takes in S-expressions as arguments instead.

In [82]:
expr = :(2 + 3 == 4 + 1)

:(2 + 3 == 4 + 1)

In [151]:
@show typeof(expr) expr.head expr.args;

typeof(expr) = Expr
expr.head = :call
expr.args = Any[:(==), :(2 + 3), :(4 + 1)]


The syntax for creating our own macro is straightforward.

In [152]:
macro myassert(expr)
    if typeof(expr) == Expr && expr.head == :call && expr.args[1] == :(==)
        return :(println("$(expr) is $($(expr))"))
    else
        return :(println("mai la"))
    end
end

@myassert (macro with 1 method)

In [153]:
@myassert 2 + 3 == 1 + 4

2 + 3 == 4 + 1 is true


In [154]:
@myassert "I luv prata"

mai la


In [155]:
@myassert 4 < 5

mai la


## Straightforward concurrency and parallelism

What's better to show existence of concurrency than sleep sort?

In [89]:
array = [2, 4, 3, 2, 3, 1, 1, 4];

In [90]:
c = Channel{Int}(0)

for value in array
    @schedule (sleep(value * 0.1); put!(c, value))
end

[take!(c) for _ in array]

8-element Array{Int64,1}:
 1
 1
 2
 2
 3
 3
 4
 4

Also for the bigger parallelizable problems there is true parallelization.

In [91]:
@show nprocs() nworkers();
addprocs(3)
@show nprocs() nworkers();

nprocs() = 1
nworkers() = 1
nprocs() = 4
nworkers() = 3


In [92]:
@everywhere using Primes

In [93]:
sum(prime(i) for i in 1:3000)
@time sum(prime(i) for i in 1:3000)

  4.087993 seconds (5.92 M allocations: 90.339 MiB, 0.19% gc time)


38645211

In [94]:
@parallel (+) for i in 1:3000
    prime(i)
end

@time @parallel (+) for i in 1:3000
    prime(i)
end

  2.479221 seconds (3.17 k allocations: 185.472 KiB)


38645211

## Demo

Did this little program one afternoon by the pool brogramming with wife =P

It converts between various bitstrings of various bitsizes as fast as possible for crypto use. To achieve that, I can either write many simple almost-repetitive functions (if I have $n$ bitsizes, I will write $O(n^2)$ functions), or make use of reflection to reduce code.

Compile languages aims to be slim and seldom have the reflection required to write this tersely. Dynamic languages tend to lack type consciousness to generate efficient code from the reflection done. I first wrote it in C++ years ago and it was madness.

Let's see how Julia deals with the struggles.

In [95]:
Word = Union{UInt8, UInt16, UInt32, UInt64}

Union{UInt16, UInt32, UInt64, UInt8}

I want an easy way to read in a word(which is a hexadecimal strings). It will be nice if I can do something as simple as

```
value = 0x0123456789abcdef0011223344556677  # cannot read in directly if system is not 128-bit
```

compared to the natural way to read

```
value = [0x01, 0x23, 0x45, 0x67, 0x89, 0xab, 0xcd, 0xef, 0x00, 0x11, 0x22, 0x33, 0x44, 0x55, 0x66, 0x77]
```

so that the crypto test vectors (usually 256-bit, 512-bit, or even longer) can be easily read in code.

In [156]:
hex2bytes("0123456789abcdef0011223344556677")

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [96]:
macro x_str(str)
    return hex2bytes(str)
end

@x_str (macro with 1 method)

In [160]:
# Example
value = x"0123456789abcdef0011223344556677"

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [98]:
num_bytes(::Type{UInt8}) = 1
num_bytes(::Type{UInt16}) = 2
num_bytes(::Type{UInt32}) = 4
num_bytes(::Type{UInt64}) = 8

num_bits(::Type{UInt8}) = 8
num_bits(::Type{UInt16}) = 16
num_bits(::Type{UInt32}) = 32
num_bits(::Type{UInt64}) = 64

mask(::Type{UInt8}) = 0xff
mask(::Type{UInt16}) = 0xffff
mask(::Type{UInt32}) = 0xffffffff
mask(::Type{UInt64}) = 0xffffffffffffffff

mask (generic function with 4 methods)

In [99]:
@generated function Base.convert(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    if num_bytes(T1) > num_bytes(T2)
        return :(pack(Vector{T1}, v))
    elseif num_bytes(T1) < num_bytes(T2)
        return :(unpack(Vector{T1}, v))
    else
        return :(v)
    end
end


function pack(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    width_result = num_bytes(T1) ÷ num_bytes(T2)
    length_result = length(v) ÷ width_result

    result = zeros(T1, length_result)

    k = 1
    for i in 1:length_result
        value = zero(T1)
        for j in 1:width_result
            value <<= num_bits(T2)
            value |= v[k]
            k += 1
        end
        result[i] = value
    end

    return result
end

function unpack(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    width_source = num_bytes(T2) ÷ num_bytes(T1)
    length_result = length(v) * width_source

    result = zeros(T1, length_result)

    i = length_result
    k = length(v)
    while i > 0
        value = v[k]
        for j in 1:width_source
            result[i] = value & mask(T1)
            value >>= num_bits(T1)
            i -= 1
        end
        k -= 1
    end

    return result
end

unpack (generic function with 1 method)

In [161]:
s = convert(Vector{UInt32}, x"0123456789abcdef0011223344556677")

4-element Array{UInt32,1}:
 0x01234567
 0x89abcdef
 0x00112233
 0x44556677

In [162]:
convert(Vector{UInt8}, s)

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [163]:
@code_llvm convert(Vector{UInt8}, x"0123456789abcdef")


define i8** @julia_convert_66537(i8**, i8** dereferenceable(40)) #0 !dbg !5 {
top:
  ret i8** %1
}


In [164]:
@code_llvm convert(Vector{UInt32}, x"0123456789abcdef")


define i8** @julia_convert_65847(i8**, i8** dereferenceable(40)) #0 !dbg !5 {
top:
  %2 = call i8** @julia_pack_65848(i8** inttoptr (i64 4491752688 to i8**), i8** nonnull %1)
  ret i8** %2
}
